# Import Libraries

In [1]:
import pandas as pd
import numpy as np


## Load URLs

# *** Download CSVs

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
# run csv files for IMDB information
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [4]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [5]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

## Specifications
Your stakeholder only wants you to include information for movies based on the following specifications:

- [ ] Exclude any movie with missing values for genre or runtime
- [ ] Include only full-length movies (titleType = "movie").
- [ ] Include only fictional movies (not from documentary genre)
- [ ] Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- [ ] Include only movies that were released in the United States


## Deliverable
After filtering out movies that do not meet the stakeholder's specifications:

- Before saving, run a final .info() for each of the dataframes to show a summary of how many movies remain and the datatypes of each feature
- Save each file to a compressed csv file "Data/" folder inside your repository.
- Commit your changes to your repository in GitHub desktop and Publish repository / Push Changes.
- Submit the link to your repository

## Handling \N Placeholder Values
According to the data dictionary, null values have been encoding as \N.
- You will want to find those and replace them with np.nan.
- However, the backslash (\) character is a special one that tells the computer to ignore whatever character comes next.
    - So if we were to say df.replace({'\N':np.nan}), the computer would see \N as an empty string.
    - To fix this, add a second backslash character, which will tell the computer that you actually WANTED to use a literal \.
    - df.replace({'\\N':np.nan})
    - Don't forget to make these replacements permanent!

## Required Preprocessing - Details
- Filtering/Cleaning Steps:
    - Title Basics:
        - [x] Replace "\N" with np.nan
        - [x] Eliminate movies that are null for runtimeMinutes
        - Eliminate movies that are null for genre
        - keep only titleType==Movie
        - keep startYear 2000-2022
        - Eliminate movies that include "Documentary" in genre (see tip below)
        - Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
- AKAs:
        - Keep only US movies.
        - Replace "\N" with np.nan
- Ratings:
        - Replace "\N" with np.nan (if any)
        - Keep only US movies (Use AKAs table, see 
        "Filtering one dataframe based on another" section below)

### Tip: Excluding Documentaries

To filter out documentaries, you will need to check if the movie's value in the Genre column contains the word documentary. **(Instead of =='documentary')**
You will also want to use the **~** operator to take the inverse of your Trues/Falses.
Example:

```
# Exclude movies that are included in the documentary category.
is_documentary = df['genres'].str.contains('documentary',case=False)
df = df[~is_documentary]
```

### Filtering One Dataframe Based On Another
Next you will filter the basics df to only include the movies that are present in your filter akas dataframe. This is how you will ultimately be able to filter the movies by region being in the US.

Here is how you can achieve this:

```
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers
```
---
Now filter basics:

```
basics = basics[keepers]
basics
```

## LOAD DATA HEADS

In [6]:
# checking basics null values
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            10
dtype: int64

There is 32 missing values in the basics file

In [7]:
# checking akas null values
akas.isna().sum()

titleId              0
ordering             0
title                5
region             108
language             0
types                0
attributes           0
isOriginalTitle      0
dtype: int64

There is 113 missing values in the akas file

In [8]:
# checking titles null values
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

There is 0 missing values in the titles file

## BASICS PREPROCESSING

In [9]:
# replacing missing values with nan
basics.replace({'\\N':np.nan}, inplace=True)

In [10]:
# double checking nan value counts
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1289706
endYear           9474850
runtimeMinutes    6780324
genres             434423
dtype: int64

In [11]:
# dropping sub-columns that 
basics = basics.dropna(subset=['runtimeMinutes', 'genres', 'startYear'])

In [12]:
# double check null value counts
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear               0
endYear           2518122
runtimeMinutes          0
genres                  0
dtype: int64

Can keep na values for endYear currently,
Replace akas nan values

In [13]:
basics['titleType'].value_counts()

tvEpisode       1213361
short            583221
movie            369880
video            178038
tvMovie           90392
tvSeries          88143
tvSpecial         17576
tvMiniSeries      16662
tvShort            8552
videoGame           317
Name: titleType, dtype: int64

### Include only full-length movies

In [14]:
# creating a filter for only movie types
movie_filter = basics['titleType']=="movie"

In [15]:
basics = basics.loc[movie_filter]

In [16]:
# sanity check each filter
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


In [17]:
basics['titleType'].value_counts()

movie    369880
Name: titleType, dtype: int64

### Year Filtering 2000-2021

In [18]:
# Change dtype to float
#basics['startYear'] = basics['startYear'].astype('float')

In [19]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


In [20]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369880 entries, 8 to 9576767
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          369880 non-null  object
 1   titleType       369880 non-null  object
 2   primaryTitle    369880 non-null  object
 3   originalTitle   369880 non-null  object
 4   isAdult         369880 non-null  object
 5   startYear       369880 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  369880 non-null  object
 8   genres          369880 non-null  object
dtypes: object(9)
memory usage: 28.2+ MB


In [21]:
# Change'startYear' to float
basics['startYear'] = basics['startYear'].astype('float')

In [22]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369880 entries, 8 to 9576767
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          369880 non-null  object 
 1   titleType       369880 non-null  object 
 2   primaryTitle    369880 non-null  object 
 3   originalTitle   369880 non-null  object 
 4   isAdult         369880 non-null  object 
 5   startYear       369880 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  369880 non-null  object 
 8   genres          369880 non-null  object 
dtypes: float64(1), object(8)
memory usage: 28.2+ MB


In [23]:
# Create filters with startYear 2000, and endYear 2022
# Keep start year 2000-2021
year_filter2000_2022 = (basics['startYear']>=2000) & (basics['startYear']<=2022)

In [24]:
basics = basics[year_filter2000_2022]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13082,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021.0,NaN,133,Documentary
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
76059,tt0077684,movie,Histórias de Combóios em Portugal,Histórias de Combóios em Portugal,0,2022.0,NaN,46,Documentary


### *** Eliminate movies that include "Documentary" (basics)

In [25]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('Documentary',case=False)
basics = basics[~is_documentary]

In [26]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001.0,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


## AKAS PREPROCESSING

In [27]:
# Replace akas null values
akas.replace({'\\N':np.nan}, inplace=True)

In [28]:
# US filter
us_filter = akas['region'] == 'US'

In [29]:
akas = akas[us_filter]
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


## Keepers Filtering (basics to akas)

In [30]:
keepers = basics['tconst'].isin(akas['titleId'])
keepers

34803       True
61116       True
67669       True
77964      False
86801       True
           ...  
9576490     True
9576499     True
9576538    False
9576583     True
9576667    False
Name: tconst, Length: 146322, dtype: bool

In [31]:
# Now filter basics
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
9575954,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
9576350,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9576490,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
9576499,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


## RATINGS PREPROCESSING

In [32]:
# replace titles null values
ratings.replace({'\\N':np.nan}, inplace=True)

# New Data Files

In [38]:
# saving compressed csv.gz files

basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)
akas.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)
ratings.to_csv("Data/title.ratings.tsv.gz", compression='gzip', index=False)

In [40]:
# Confirm saved file can be previewed
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
